In [1]:
# Clone the repo (choose the correct branch)
!git clone --branch task-3 https://github.com/sumeyaaaa/analysis-of-financial-news-data.git


^C


Cloning into 'analysis-of-financial-news-data'...


In [ ]:
from google.colab import files

uploaded = files.upload()  # This opens a file picker. Select your CSV(s).


Saving news_yfinance_merged.csv to news_yfinance_merged (1).csv


In [ ]:
import pandas as pd

df = pd.read_csv('news_yfinance_merged.csv')
print(df.head())


   Unnamed: 0                                           headline  \
0        7120  Tech Stocks And FAANGS Strong Again To Start D...   
1        7121      10 Biggest Price Target Changes For Wednesday   
2        7122  Benzinga Pro's Top 5 Stocks To Watch For Wed.,...   
3        7123  Deutsche Bank Maintains Buy on Apple, Raises P...   
4        7124  Apple To Let Users Trade In Their Mac Computer...   

                                                 url          publisher  \
0  https://www.benzinga.com/government/20/06/1622...         JJ Kinahan   
1  https://www.benzinga.com/analyst-ratings/price...         Lisa Levin   
2  https://www.benzinga.com/short-sellers/20/06/1...  Benzinga Newsdesk   
3  https://www.benzinga.com/news/20/06/16219873/d...  Benzinga Newsdesk   
4  https://www.benzinga.com/news/20/06/16218697/a...      Neer Varshney   

         date stock      Close       Open       High        Low     Volume  
0  2020-06-10  AAPL  88.209999  86.974998  88.692497  86.522499

In [ ]:
!pip install textblob
from textblob import TextBlob


In [ ]:
# For VADER (NLTK)
import pandas as pd
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer




[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
vader = SentimentIntensityAnalyzer()

def vader_sentiment(text):
    if pd.isnull(text):
        return [0, "neutral"]
    score = vader.polarity_scores(str(text))['compound']
    if score >= 0.1:
        sentiment = "positive"
    elif score <= -0.1:
        sentiment = "negative"
    else:
        sentiment = "neutral"
    return [score, sentiment]

df[['vader_score', 'vader_sentiment']] = df['headline'].apply(vader_sentiment).apply(pd.Series)


In [ ]:
def textblob_sentiment(text):
    if pd.isnull(text):
        return [0, "neutral"]
    blob = TextBlob(str(text))
    score = blob.sentiment.polarity
    if score > 0.1:
        sentiment = "positive"
    elif score < -0.1:
        sentiment = "negative"
    else:
        sentiment = "neutral"
    return [score, sentiment]

df[['tb_score', 'tb_sentiment']] = df['headline'].apply(textblob_sentiment).apply(pd.Series)


In [ ]:
# Preview
df[['headline', 'vader_score', 'vader_sentiment', 'tb_score', 'tb_sentiment']].head()

# Save to new CSV
df.to_csv('/content/analysis-of-financial-news-data/news_with_sentiment.csv', index=False)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install transformers --quiet
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load FinBERT
tokenizer = AutoTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = AutoModelForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

def finbert_sentiment(text):
    if pd.isnull(text):
        return "neutral"
    inputs = tokenizer(str(text), return_tensors="pt", truncation=True, max_length=64)
    with torch.no_grad():
        logits = model(**inputs).logits
    sentiment_id = logits.argmax().item()
    sentiment = model.config.id2label[sentiment_id]
    return sentiment


# Apply (this can take a bit of time for 13k rows)
df['finbert_sentiment'] = df['headline'].apply(finbert_sentiment)

# Save result
df.to_csv('/content/analysis-of-financial-news-data/news_with_finbert_sentiment.csv', index=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
from google.colab import files
files.download('/content/analysis-of-financial-news-data/news_with_finbert_sentiment.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>